In [6]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN
from aif360.datasets import BinaryLabelDataset
from aif360.metrics import ClassificationMetric

In [8]:
# Load the data
train = pd.read_csv(r'../Resources/Data/baseline_train_data.csv')
test = pd.read_csv(r'../Resources/Data/baseline_test_data.csv')
valid = pd.read_csv(r'../Resources/Data/baseline_valis_data.csv')

# Define target and features
target = 'two_year_recid'

# Separate features and target for training, testing, and validation data
X_train = train.drop(columns=[target])
y_train = train[target]

X_test = test.drop(columns=[target])
y_test = test[target]

X_valid = valid.drop(columns=[target])
y_valid = valid[target]

# Define sampling strategies
over_sampler = SMOTE(random_state=42)
under_sampler = RandomUnderSampler(random_state=42)
combined_sampler = SMOTEENN(random_state=42)

In [10]:
# Function to train and evaluate the model with different sampling techniques
def evaluate_fairness(X_train, y_train, sampling_strategy=None):
    if sampling_strategy:
        X_train, y_train = sampling_strategy.fit_resample(X_train, y_train)
    
    log_reg = LogisticRegression(max_iter=1000)
    log_reg.fit(X_train, y_train)
    
    # Predict on the test data
    y_pred_test = log_reg.predict(X_test)
    
    # Convert to AIF360 BinaryLabelDataset format
    test_aif = BinaryLabelDataset(df=pd.concat([X_test, y_test], axis=1),
                                  label_names=[target],
                                  protected_attribute_names=['sex'])
    
    test_aif_pred = BinaryLabelDataset(df=pd.concat([X_test, pd.Series(y_pred_test, name=target)], axis=1),
                                       label_names=[target],
                                       protected_attribute_names=['sex'])
    
    # Define privileged and unprivileged groups
    privileged_groups = [{'sex': 1}]
    unprivileged_groups = [{'sex': 0}]
    
    # Compute fairness metrics
    metric_test_pred = ClassificationMetric(test_aif, test_aif_pred,
                                            unprivileged_groups=unprivileged_groups,
                                            privileged_groups=privileged_groups)
    
    stat_parity_diff = metric_test_pred.statistical_parity_difference()
    avg_odds_diff = metric_test_pred.average_odds_difference()
    equal_opp_diff = metric_test_pred.equal_opportunity_difference()
    
    # Display results
    print(f"Sampling Strategy: {sampling_strategy.__class__.__name__ if sampling_strategy else 'None'}")
    print(f"Statistical Parity Difference: {stat_parity_diff}")
    print(f"Average Odds Difference: {avg_odds_diff}")
    print(f"Equal Opportunity Difference: {equal_opp_diff}")
    print()

In [12]:
# Evaluate the model with different sampling strategies
print("With Over-Sampling (SMOTE):")
evaluate_fairness(X_train, y_train, over_sampler)

print("With Under-Sampling (RandomUnderSampler):")
evaluate_fairness(X_train, y_train, under_sampler)

print("With Combined Sampling (SMOTE + ENN):")
evaluate_fairness(X_train, y_train, combined_sampler)

With Over-Sampling (SMOTE):
Sampling Strategy: SMOTE
Statistical Parity Difference: 0.32795719619692826
Average Odds Difference: 0.315755227048678
Equal Opportunity Difference: 0.3286319394376352

With Under-Sampling (RandomUnderSampler):
Sampling Strategy: RandomUnderSampler
Statistical Parity Difference: 0.3423341930020401
Average Odds Difference: 0.3300062077613293
Equal Opportunity Difference: 0.3385003604902668

With Combined Sampling (SMOTE + ENN):
Sampling Strategy: SMOTEENN
Statistical Parity Difference: 0.25447476808191233
Average Odds Difference: 0.24212005215492188
Equal Opportunity Difference: 0.24968457101658253

